In [12]:
import numpy as np
import pandas as pd

df = pd.read_csv('data.csv')

unnessary_features = ['Observation_ID', 'Update_Datetime', 'Observation_Time', 
                      'Site_ID', 'Site_Name', 
                      'Individual_ID', 
                      'Phenophase_ID', 'Phenophase_Category', 'Phenophase_Description', 
                      'Intensity_Category_ID', 'Intensity_Value', 'Abundance_Value', 
                      'Kingdom', 'Species_Category']

df = df.drop(columns=unnessary_features)
df = df[df['Day_of_Year'] <= 365]

In [ ]:
# ---------- Drop Phenophase_Status == -1 and report ----------
before = len(df)
df = df[df['Phenophase_Status'] != -1].copy()
after = len(df)
print(f"Rows before: {before}, after dropping Phenophase_Status==-1: {after}, dropped: {before-after}")

Rows before: 6503, after dropping Phenophase_Status==-1: 6487, dropped: 16


In [ ]:
# ---------- Fix -9999 error codes in climate features ----------
climate_features = ['AGDD','Tmax','Tmin','Prcp','Accum_Prcp','Daylength']
# convert -9999 -> NaN
df[climate_features] = df[climate_features].replace(-9999, np.nan)

# convert Observation_Date to datetime & create year
df['Observation_Date'] = pd.to_datetime(df['Observation_Date'])
df['year'] = df['Observation_Date'].dt.year

# create site_id from lat/lon (simple deterministic encoding)
df['site_id'] = (df['Latitude'].round(4).astype(str) + '_' + df['Longitude'].round(4).astype(str))
# Optionally map to integers:
df['site_id'] = pd.factorize(df['site_id'])[0] + 1  # site_id as ints starting at 1

# create daily mean temperature (pyPhenology examples use "temperature")
df['temperature'] = (df['Tmax'] + df['Tmin']) / 2.0

In [ ]:
# ---------- Interpolate climate missing values per site (time-series interpolation) ----------
# sort then interpolate by site
df = df.sort_values(['site_id','Observation_Date'])
df[climate_features] = df.groupby('site_id')[climate_features].transform(
    lambda g: g.interpolate(method='linear', limit_direction='both')
)
# if still NaN (edges), fill with column median (per-site median could be used)
df[climate_features] = df[climate_features].fillna(df[climate_features].median())

# also recompute temperature if it used NaNs earlier
df['temperature'] = (df['Tmax'] + df['Tmin']) / 2.0

In [ ]:
# ---------- Create observations DataFrame (first bloom DOY) ----------
# define first bloom per site-species-year: min Day_of_Year where Phenophase_Status==1
bloom_obs = (
    df[df['Phenophase_Status'] == 1]
    .groupby(['site_id','Species','year'], as_index=False)['Day_of_Year']
    .min()
    .rename(columns={'Species':'species', 'Day_of_Year':'doy'})
)
# If you prefer use Species_ID instead of species string:
# bloom_obs = bloom_obs.rename(columns={'Species_ID':'species_id', ...})

# pyPhenology example uses a 'phenophase' id; set a single id (e.g. 501 for 'flowers')
bloom_obs['phenophase'] = 501

# final observations DataFrame columns: ['species','site_id','year','doy','phenophase']
observations = bloom_obs[['species','site_id','year','doy','phenophase']]

print("Observations (first bloom) sample:")
print(observations.head())

Observations (first bloom) sample:
  species  site_id  year  doy  phenophase
0  annuus        1  2018   76         501
1  annuus        3  2018  317         501
2  annuus        4  2019  131         501
3  annuus        5  2021  117         501
4  annuus        5  2022    4         501


In [ ]:
# ---------- Create predictors DataFrame (daily temperature series) ----------
# pyPhenology predictors expected columns: at least ['site_id','temperature','year','doy']
predictors = df[['site_id','temperature','year','Day_of_Year']].rename(columns={'Day_of_Year':'doy'})
# ensure types
predictors = predictors.astype({'site_id': int, 'year': int, 'doy': float, 'temperature': float})

print("Predictors sample:")
print(predictors.head())

Predictors sample:
   site_id  temperature  year    doy
0        1        20.00  2018   76.0
1        1        20.00  2018   76.0
2        2        15.00  2018  150.0
3        2        15.00  2018  150.0
4        3        10.25  2018  317.0


In [30]:
predictors.describe()

,site_id,temperature,year,doy
count,6487.000000,6487.000000,6487.000000,6487.000000
mean,12.505164,16.553695,2019.226761,184.941730
std,2.222914,5.002215,2.216171,102.479229
min,1.000000,-3.000000,2016.000000,1.000000
25%,12.000000,12.402500,2017.000000,97.000000
50%,12.000000,16.500000,2019.000000,185.000000
75%,12.000000,20.250000,2021.000000,273.000000
max,34.000000,33.125000,2023.000000,365.000000


In [ ]:
# ---------- Split train/test (by year) ----------
years = sorted(observations['year'].unique())
# hold out the latest year as test
test_year = years[-1]
train_years = years[:-1]

obs_train = observations[observations['year'].isin(train_years)]
obs_test  = observations[observations['year'] == test_year]

preds_train = predictors[predictors['year'].isin(train_years)]
preds_test  = predictors[predictors['year'] == test_year]

print(f"Train site-year combos: {len(obs_train)}; Test site-year combos: {len(obs_test)}")

Train site-year combos: 46; Test site-year combos: 3


In [ ]:
# ---------- Train model with pyPhenology ----------
from pyPhenology import models

# ThermalTime model (standard growing-degree-days)
model = models.ThermalTime()

# Fit on training observations + predictors
# NOTE: fit may take a bit; uses scipy optimizers internally.
model.fit(obs_train, preds_train)

# fitted parameters
params = model.get_params()
print("Fitted model parameters:", params)

d:\VSCode\Python\.venv\Lib\site-packages\pyPhenology\models\utils\misc.py:58: UserWarning: Dropped temperature data for doy 1.0 due to missing data. Most likely from leap year mismatch
  warn("""Dropped temperature data for doy {d} due to missing data. Most likely from leap year mismatch""".format(d=first_doy_column))
d:\VSCode\Python\.venv\Lib\site-packages\pyPhenology\models\utils\misc.py:66: UserWarning: Dropped temperature data for doy 365.0 due to missing data. Most likely from leap year mismatch
  warn("""Dropped temperature data for doy {d} due to missing data. Most likely from leap year mismatch""".format(d=last_doy_column))
d:\VSCode\Python\.venv\Lib\site-packages\pyPhenology\models\utils\misc.py:81: UserWarning: Dropped 46 of 46 observations because of missing data
 Missing data from: 
    site_id  year
0         1  2018
1         3  2018
2         4  2019
3         5  2021
4         5  2022
5         6  2021
6         7  2022
7         9  2016
8        10  2017
10       11  

Fitted model parameters: {'t1': np.float64(-19.69325300434207), 'T': np.float64(1.9736485233480372), 'F': np.float64(824.1150140305742)}


In [ ]:
# ---------- Predict on test ----------
# prepare to_predict - unique site/species/year combos we want predictions for
to_predict = obs_test[['site_id','species','year']].drop_duplicates().reset_index(drop=True)

# pyPhenology predict API: predict(to_predict, predictors)
# try that call and fall back to predict(predictors) if needed
try:
    pred_vals = model.predict(to_predict, preds_test)
except TypeError:
    # fallback (some versions accept only predictors)
    pred_vals = model.predict(preds_test)

# pred_vals expected to be an array/list of predicted DOY for each row in to_predict
if len(pred_vals) == len(to_predict):
    to_predict['predicted_doy'] = np.array(pred_vals).reshape(-1)
else:
    print(f"Warning: Model returned {len(pred_vals)} predictions for {len(to_predict)} test cases. Filling with NaN.")
    to_predict['predicted_doy'] = np.nan

# join with observed
eval_df = to_predict.merge(
    obs_test.rename(columns={'species':'species','doy':'observed_doy'}),
    on=['site_id','species','year'],
    how='left'
)

# compute RMSE / MAE only for rows with valid predictions
from sklearn.metrics import mean_squared_error, mean_absolute_error
valid = eval_df.dropna(subset=['observed_doy', 'predicted_doy'])
if len(valid) > 0:
    rmse = np.sqrt(mean_squared_error(valid['observed_doy'], valid['predicted_doy']))
    mae  = mean_absolute_error(valid['observed_doy'], valid['predicted_doy'])
    print(f"Test RMSE = {rmse:.3f} days; MAE = {mae:.3f} days")
else:
    print("No valid predictions to evaluate (all predicted_doy are NaN).")
print(eval_df.head())


No valid predictions to evaluate (all predicted_doy are NaN).
   site_id      species  year  predicted_doy  observed_doy  phenophase
0        8       annuus  2023            NaN           223         501
1       12  californica  2023            NaN           128         501
2       13  californica  2023            NaN           115         501


d:\VSCode\Python\.venv\Lib\site-packages\pyPhenology\models\utils\misc.py:58: UserWarning: Dropped temperature data for doy 2.0 due to missing data. Most likely from leap year mismatch
  warn("""Dropped temperature data for doy {d} due to missing data. Most likely from leap year mismatch""".format(d=first_doy_column))
d:\VSCode\Python\.venv\Lib\site-packages\pyPhenology\models\utils\misc.py:66: UserWarning: Dropped temperature data for doy 365.0 due to missing data. Most likely from leap year mismatch
  warn("""Dropped temperature data for doy {d} due to missing data. Most likely from leap year mismatch""".format(d=last_doy_column))
d:\VSCode\Python\.venv\Lib\site-packages\pyPhenology\models\utils\misc.py:81: UserWarning: Dropped 3 of 3 observations because of missing data
 Missing data from: 
   site_id  year
0        8  2023
1       12  2023
2       13  2023
  warn('Dropped {n0} of {n1} observations because of missing data'.format(n0=n_dropped, n1=original_sample_size) +


In [ ]:
# ---------- Save trained model ----------
import joblib
joblib.dump(model, 'pyphenology_thermaltime_model.pkl')
print("Model saved to pyphenology_thermaltime_model.pkl")

Model saved to pyphenology_thermaltime_model.pkl
